In [37]:
import os
from dotenv import load_dotenv
from langchain_ibm import WatsonxLLM
from langchain.prompts import PromptTemplate

In [38]:
load_dotenv()
watsonx_apikey = os.getenv('WATSONX_APIKEY')
project_id =  os.getenv('PROJECT_ID')
url= os.getenv('URL')


In [39]:
# Init llm model
from langchain_ibm import WatsonxLLM
params_dic= {
    "decoding_method": "greedy",
    "temperature": 0,
    "min_new_tokens": 5,
    "max_new_tokens": 5000,
    # "stop_sequences": ["\n\n"]
}
llm = WatsonxLLM(
    model_id=
            # "meta-llama/llama-3-405b-instruct",
            # "meta-llama/llama-3-2-90b-vision-instruct",
        #    'ibm/granite-13b-instruct-v2',
           'ibm/granite-3-8b-instruct',
            
    url=url,
    apikey=watsonx_apikey,
    project_id=project_id,
    params=params_dic
)

In [ ]:
# modelos_granite_disponibles = [
#     'ibm/granite-13b-instruct-v2', 
#     'ibm/granite-20b-code-instruct', 
#     'ibm/granite-20b-multilingual', 
#     'ibm/granite-3-2-8b-instruct', 
#     'ibm/granite-3-2b-instruct', 
#     'ibm/granite-3-8b-instruct', 
#     'ibm/granite-34b-code-instruct', 
#     'ibm/granite-3b-code-instruct', 
#     'ibm/granite-8b-code-instruct', 
#     'ibm/granite-guardian-3-2b', 
#     'ibm/granite-guardian-3-8b', 
#     'ibm/granite-vision-3-2-2b'
# ]

In [4]:
llm.invoke("hello, whats your name?")

"\n\nHello! I'm an assistant developed to help answer your questions. I don't have a personal name, but you can call me Assistant. How can I assist you today?"

In [ ]:
# prompt_text="""   
# Eres un asistente especializado en el análisis de métricas de calidad del agua potable. 
# Se te ha proporcionado un historial de 10 muestras tomadas en diferentes momentos en un mismo punto de agua potable:

# "{datos_calidad_agua}"

# Analiza los datos proporcionados evaluando si el valor más reciente presenta una desviación significativa respecto a la tendencia general observada en las muestras anteriores. 

# Devuelve solo el resumen de tu evaluación con una longitud de 1000 tokens aproximadamente.

# Respuesta:
# """

In [ ]:
prompt_text="""  

Rol:
Eres un asistente experto en el análisis de métricas de calidad del agua potable.

Contexto:
Se te ha proporcionado un historial de 10 muestras tomadas en diferentes fechas en un mismo punto de agua potable:

Datos de calidad del agua:
{datos_calidad_agua}  

Tarea:

Analiza las tendencias generales de los parámetros a lo largo del tiempo.

Evalúa la muestra más reciente y determina si presenta una desviación significativa respecto a la tendencia previa.

Identifica posibles riesgos para la calidad del agua, indicando qué parámetros están fuera de los valores normales.

Si la calidad ha mejorado o empeorado, explica las posibles causas y sus implicaciones.

Salida esperada:
Resumen de evaluación (≈1000 tokens):

Estado actual de la calidad del agua en la última medición.

Comparación con tendencias anteriores y valores de referencia.

Identificación de parámetros críticos con explicaciones.

Conclusión sobre la estabilidad de la calidad del agua.
"""

In [ ]:
# promt_text= """    
# Eres un asistente experto en analizar metricas de calidad de agua. 
# Te han proporcionado el historial (10 muestras en diferentes moemntos)de datos de calidad de agua en un mismo punto de agua potable:

# "{datos_calidad_agua}"

# Tu tarea es analizar los datos y ver si el valor mas reciente se desvía de la tendencia general de los datos.
# Dame una evaluación de la evolución y del estado actual de la calidad del agua en este punto de agua potable.
# Respuesta:
# """

In [58]:
def ask_llm(llm, datos_calidad):
    # Crear el prompt con LangChan
    prompt_template = PromptTemplate(
            input_variables=["datos_calidad_agua"],
            template=prompt_text,
    )

    # 🔍 Print del prompt generado con los datos: 
    prompt_final = prompt_template.format(
                                    datos_calidad_agua=datos_calidad
    )
    print("\n" + "="*50)
    print("🔍 PROMPT GENERADO:")
    print(prompt_final)
    print("="*50 + "\n")

    
    
    llm_chain = prompt_template | llm
    output = llm_chain.invoke({"datos_calidad_agua": datos_calidad}) 
    return output

    

# datos en sampling_point_id = "48003090003"

In [22]:
import requests
import pandas as pd

# Definir los parámetros fijos
sampling_point_id = "48003090003"
lang = "SPANISH"

# Lista de fechas
measurement_dates = [
    "2025-01-03", "2024-12-27", "2024-12-17", "2024-12-13",
    "2024-11-05", "2024-10-25", "2024-10-01", "2024-09-27",
    "2024-09-17", "2024-09-13"
]

# Crear una lista para almacenar los resultados
data_list = []

# Iterar sobre cada fecha y hacer la solicitud
for date in measurement_dates:
    url = f"https://api.euskadi.eus/water-quality/sampling-points/{sampling_point_id}/measurements/{date}/analytical-data"
    params = {"lang": lang}
    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        row = {"date": date}
        
        # Agregar cada parámetro como columna
        for entry in data:
            row[entry["parameter"]] = entry["value"]
        
        data_list.append(row)
    else:
        print(f"Error con la fecha {date}: {response.status_code}")

# Convertir los datos en un DataFrame
df = pd.DataFrame(data_list)

# Mostrar el DataFrame
print(df)

# Guardar en un archivo CSV
# df.to_csv("analytical_data.csv", index=False)


         date  ALUMINIO  AMONIO  CLORO LIBRE RESIDUAL  \
0  2025-01-03       0.0     0.0                  0.51   
1  2024-12-27       0.0     0.0                  0.48   
2  2024-12-17       0.0     0.0                  0.53   
3  2024-12-13       0.0     0.0                  0.49   
4  2024-11-05       0.0     0.0                  0.42   
5  2024-10-25       0.0     0.0                  0.51   
6  2024-10-01      50.0     0.1                  0.50   
7  2024-09-27      50.0     0.1                  0.47   
8  2024-09-17      50.0     0.1                  0.45   
9  2024-09-13      50.0     0.1                  0.52   

   CLOSTRIDIUM PERFRINGENS (INCLUIDAS LAS ESPORAS)  COLIFORMES TOTALES (NMP)  \
0                                              0.0                       0.0   
1                                              0.0                       0.0   
2                                              0.0                       0.0   
3                                              0.0  

In [26]:
datos_calidad_= df.to_string()

In [29]:
print(datos_calidad_)

         date  ALUMINIO  AMONIO  CLORO LIBRE RESIDUAL  CLOSTRIDIUM PERFRINGENS (INCLUIDAS LAS ESPORAS)  COLIFORMES TOTALES (NMP)  COLOR (Pt/Co)  CONDUCTIVIDAD (20ºC)  ENTEROCOCOS INTESTINALES (NMP)  ESCHERICHIA COLI (NMP)  NITRITOS  PH (20ºC)  RECUENTO DE BACTERIAS HETEROTROFAS A 22ºC  TEMPERATURA  TURBIDEZ
0  2025-01-03       0.0     0.0                  0.51                                              0.0                       0.0            0.0                 177.0                             0.0                     0.0      0.00        8.1                                        1.0         10.6      0.00
1  2024-12-27       0.0     0.0                  0.48                                              0.0                       0.0            0.0                 186.0                             0.0                     0.0      0.00        8.1                                        0.0         10.6      0.00
2  2024-12-17       0.0     0.0                  0.53                    

# respuesta llm

In [59]:
respuesta=ask_llm(llm, datos_calidad_)


🔍 PROMPT GENERADO:
  

Rol:
Eres un asistente experto en el análisis de métricas de calidad del agua potable.

Contexto:
Se te ha proporcionado un historial de 10 muestras tomadas en diferentes fechas en un mismo punto de agua potable:

Datos de calidad del agua:
         date  ALUMINIO  AMONIO  CLORO LIBRE RESIDUAL  CLOSTRIDIUM PERFRINGENS (INCLUIDAS LAS ESPORAS)  COLIFORMES TOTALES (NMP)  COLOR (Pt/Co)  CONDUCTIVIDAD (20ºC)  ENTEROCOCOS INTESTINALES (NMP)  ESCHERICHIA COLI (NMP)  NITRITOS  PH (20ºC)  RECUENTO DE BACTERIAS HETEROTROFAS A 22ºC  TEMPERATURA  TURBIDEZ
0  2025-01-03       0.0     0.0                  0.51                                              0.0                       0.0            0.0                 177.0                             0.0                     0.0      0.00        8.1                                        1.0         10.6      0.00
1  2024-12-27       0.0     0.0                  0.48                                              0.0               

In [60]:
print(respuesta)


Análisis de tendencias:

La serie temporal de las muestras muestra una variabilidad en los parámetros de calidad del agua. El aluminio, el amonio, los nitratos, el recuento de bacterias heterotrofas a 22ºC, el pH y la temperatura se mantienen relativamente estables, con valores cercanos a cero o dentro de los rangos normales. Sin embargo, el cloro libre residual, la conductividad, el recuento de enterococos intestinales, el escherichia coli, la turbididad y el recuento total de coliformes se han visto afectados por fluctuaciones.

Evaluación de la última muestra:

La última muestra, tomada el 2025-01-03, presenta valores de cloro libre residual de 0.51 mg/L, conductividad de 177 µS/cm, recuento de enterococos intestinales de 0.0 NMP, escherichia coli de 0.0 NMP, turbididad de 1.0 NTU y recuento total de coliformes de 0.0 NMP. Estos valores están dentro de los rangos normales y no muestran una desviación significativa respecto a las tendencias anteriores.

Riesgos para la calidad del a

In [48]:
print(respuesta)


El análisis de los datos proporcionados revela una tendencia general en la calidad del agua en el punto de agua potable en cuestión. A continuación, se presenta un resumen detallado de los hallazgos:

1. Aluminio: El valor más reciente (0.0) no presenta aluminio, lo que es consistente con las muestras anteriores.
2. Amonio: El valor más reciente (0.0) no presenta amonio, lo que es consistente con las muestras anteriores.
3. Cloro libre residual: El valor más reciente (0.51) es ligeramente superior al promedio de las muestras anteriores (0.48), pero aún se mantiene dentro del rango aceptable.
4. Clostridium perfringens (incluidas las esporas): El valor más reciente (0.0) no presenta Clostridium perfringens, lo que es consistente con las muestras anteriores.
5. Coliformes totales (NMP): El valor más reciente (0.0) no presenta coliformes totales, lo que es consistente con las muestras anteriores.
6. Color (Pt/Co): El valor más reciente (0.0) no presenta color, lo que es consistente con l

In [55]:
print(respuesta)


El análisis de las muestras de agua potable revela una tendencia general de estabilidad en los parámetros de calidad, con algunas variaciones en los niveles de aluminio, amonio, cloro libre residual, enterococos intestinales, Escherichia coli, nitritos y recuento de bacterias heterotrofas a 22ºC.

En cuanto a los niveles de aluminio, se observa un aumento significativo en la muestra del 6 de octubre, con un valor de 50.0 ppm, seguido de una disminución en las muestras posteriores. Sin embargo, el valor más reciente, correspondiente a la muestra del 13 de septiembre, muestra un nivel de aluminio de 50.0 ppm, lo que indica una continuidad en la tendencia observada en las muestras anteriores.

Los niveles de amonio y cloro libre residual se mantienen constantes en valores bajos, con variaciones mínimas entre las muestras.

En cuanto a los enterococos intestinales y Escherichia coli, se observa un valor de 0.0 NMP en las mayoría de las muestras, con excepción de la del 6 de octubre, donde